# OpenDRIVE - Lanelet2 conversion

### 1. Imports

In [1]:
import os
import shutil
from lxml import etree
from crdesigner.common.config.lanelet2_config import lanelet2_config
from crdesigner.map_conversion.lanelet2.cr2lanelet import CR2LaneletConverter
from commonroad.scenario.scenario import Location, GeoTransformation
from crdesigner.map_conversion.map_conversion_interface import opendrive_to_commonroad
from pathlib import Path

### 2. Declarations

In [ ]:
# Input handling
input_dir = Path("./sample_data")
set_list = [
    "naive",
    "CARLA",
    "esmini"
]

# Output handling
output_dir = Path("./output/old")
if output_dir.exists():
    print(f"Output dir {output_dir} exists. Purging...")
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok = True)

# Georeference params
georeference_string = "EPSG:3857"
x_translation = 0.0 # 658761.0
y_translation = 0.0 # 4542599.0

# Scenario handling
location_geotransformation = GeoTransformation(
    georeference_string,
    x_translation = x_translation,
    y_translation = y_translation,
    z_rotation = None,
    scaling = None
)
scenario_location = Location(
    geo_name_id = 11,
    gps_latitude = 0.0,
    gps_longitude = 0.0,
    geo_transformation = location_geotransformation,
    environment = None
)

### 3. Attempt conversion

In [ ]:
for set_name in set_list:
    print(f"\n=============== Working on {set_name} ===============\n")
    this_set_input_path = input_dir / set_name
    this_set_output_path = output_dir / set_name
    if not (os.path.exists(this_set_output_path)):
        os.makedirs(this_set_output_path)

    for input_file in os.listdir(this_set_input_path):
        input_file_path = this_set_input_path / input_file
        try:
            # Input handling
            print(f"Converting {input_file_path}")

            # Output handling
            input_file_tail_trimmed = ".".join(input_file.split(".")[ : -1])
            output_filename = f"converted_{input_file_tail_trimmed}.osm"
            output_path = this_set_output_path / output_filename

            # Scenario initialization
            scenario = opendrive_to_commonroad(input_file_path)
            scenario.location = scenario_location
            
            # Conversion
            if (scenario):
                l2osm = CR2LaneletConverter(lanelet2_config)
                osm = l2osm(scenario)

                with open(output_path, "wb") as file_out:
                    file_out.write(etree.tostring(
                        osm, 
                        xml_declaration = True, 
                        encoding = "UTF-8", 
                        pretty_print = True
                    ))

        except Exception as e:
            print(f"Error: {e}")
            continue